# S2W7D5: 模型落地与推理 (Inference & Deployment)

今天是一个里程碑。我们要彻底抛弃沉重的 `Trainer`，像一个真正的**部署工程师**一样，写出一个轻量级、高性能的推理引擎。

这不仅仅是代码，更是**具身智能系统的“感官接口”**。你的机器人本体（比如基于 ROS 的系统）将通过这个接口调用你的模型，把听到的指令转化为行动。

## 1 🏗️ 核心架构：解耦推理引擎

在训练时，我们需要 `Trainer` 帮我们算梯度、存 Checkpoint。
但在推理（Inference）时，我们只需要：**Tokenizer + Model权重 + 前向传播**。

我们要编写一个独立的 `IntentPredictor` 类，封装所有细节。

请在 `src/` 目录下新建文件 **`inference.py`**。

**文件路径**: `project_root/src/inference.py`

```python
import torch
import torch.nn.functional as F
from transformers import BertTokenizer
from safetensors.torch import load_file
from src.models.model_bert import BertClassifier
from src.config import PRETRAINED_MODEL_DIR

class IntentPredictor:
    def __init__(self, checkpoint_path, num_labels=3):
        """
        初始化推理引擎
        :param checkpoint_path: 训练好的模型权重路径 (包含 model.safetensors)
        :param num_labels: 类别数量
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.label_map = {0: "MOVE", 1: "STOP", 2: "GRAB"} # 这里的映射要和训练时一致
        
        print(f"正在加载推理资源... (Device: {self.device})")
        
        # 1. 加载分词器 (从原始预训练目录加载 vocab)
        self.tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_DIR)
        
        # 2. 初始化模型架构
        self.model = BertClassifier(model_path=PRETRAINED_MODEL_DIR, num_labels=num_labels)
        
        # 3. 加载训练好的权重 (.safetensors)
        # 这里的 checkpoint_path 应该是类似 output/results_weighted/checkpoint-500
        weight_path = f"{checkpoint_path}/model.safetensors"
        try:
            state_dict = load_file(weight_path)
            self.model.load_state_dict(state_dict)
            print("✅ 权重加载成功！")
        except Exception as e:
            print(f"❌ 权重加载失败: {e}")
            raise e
            
        # 4. 关键：切换到评估模式 & 移动到 GPU
        self.model.to(self.device)
        self.model.eval() 

    def predict(self, text):
        """
        单条文本预测
        :param text: 用户指令字符串 "把杯子拿起来"
        :return: 字典 {intent, confidence, latency}
        """
        # 1. 预处理 (Tokenize)
        inputs = self.tokenizer(
            text, 
            return_tensors="pt", 
            padding=True, 
            truncation=True, 
            max_length=128
        )
        
        # 2. 搬运数据到 Device
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        # 3. 推理 (Inference)
        with torch.no_grad(): # ★ 必须关掉梯度，节省显存并加速
            logits = self.model(**inputs)
            
        # 4. 后处理 (Post-processing)
        # Logits -> Probabilities (Softmax)
        probs = F.softmax(logits, dim=-1) 
        
        # 获取最大概率和对应的 ID
        confidence, pred_id = torch.max(probs, dim=-1)
        
        pred_id = pred_id.item()
        confidence = confidence.item()
        
        return {
            "text": text,
            "intent": self.label_map[pred_id], # 转化为人类可读标签
            "confidence": f"{confidence:.4f}",
            "intent_id": pred_id
        }
```


## 2 💻 交互式 Demo：指挥你的机器人

现在回到 Notebook `S2W7D5_Inference_Deployment.ipynb`，我们来模拟一个真实的控制台。

In [1]:
import sys
sys.path.append("../../")
from src.inference import IntentPredictor
# ★ 引入 PROJECT_ROOT，它是项目根目录的绝对路径，最稳健
from src.config import CHECKPOINT_DIR 

# 1. 指定你的最佳 Checkpoint 路径
# 我们利用 pathlib 的路径拼接功能，指向刚才保存的 final_best
# 路径逻辑: 项目根目录 -> output -> train -> weighted_model -> final_best
best_ckpt_path = str(CHECKPOINT_DIR / "final_best")

print(f"正在加载模型路径: {best_ckpt_path}")

# 2. 实例化推理引擎
# 注意：如果报错说路径不存在，请去左侧文件栏确认一下 output 文件夹到底是在根目录还是在当前目录下
try:
    predictor = IntentPredictor(checkpoint_path=best_ckpt_path, num_labels=3)
except Exception as e:
    print("\n⚠️ 路径加载失败？可能是因为训练时的保存路径是相对路径。")
    print("请尝试将 best_ckpt_path 改为: './output/train/weighted_model/final_best'")
    raise e

# 3. 模拟具身智能控制台
print("\n" + "="*40)
print("🤖 具身智能指令解析系统 v2.0 (Weighted)")
print("输入 'q' 退出")
print("="*40)
 
while True:
    # 模拟接收语音转文字的输入
    user_input = input("\n🗣️ 请输入指令: ")
    
    if user_input.lower() == 'q': 
        print("系统下线。Bye!")
        break
    
    if not user_input.strip():
        continue
        
    # 调用预测
    result = predictor.predict(user_input)
    
    # 格式化输出结果
    intent = result['intent']
    conf = float(result['confidence'])
    
    # 模拟机器人决策逻辑
    action_log = ""
    if conf < 0.6:
        action_log = "⚠️ 置信度过低，请求重复指令..."
    elif intent == "STOP":
        action_log = "🛑 执行紧急制动！"
    elif intent == "GRAB":
        action_log = "🦾 启动机械臂抓取程序..."
    elif intent == "MOVE":
        action_log = "?? 底盘开始移动..."
        
    print(f"  [解析结果] 意图: {intent} | 置信度: {conf:.2%}")
    print(f"  [机器人动作] {action_log}")

正在加载模型路径: /home/goodminton/study/AI-Interview-Sprint/output/train/checkpoints/final_best
正在加载推理资源... (Device: cuda)
✅ 权重加载成功！(已忽略多余的 Loss 权重)

🤖 具身智能指令解析系统 v2.0 (Weighted)
输入 'q' 退出
  [解析结果] 意图: MOVE | 置信度: 74.95%
  [机器人动作] ?? 底盘开始移动...
  [解析结果] 意图: GRAB | 置信度: 48.24%
  [机器人动作] ⚠️ 置信度过低，请求重复指令...
  [解析结果] 意图: STOP | 置信度: 60.89%
  [机器人动作] 🛑 执行紧急制动！
  [解析结果] 意图: MOVE | 置信度: 51.49%
  [机器人动作] ⚠️ 置信度过低，请求重复指令...
系统下线。Bye!


**任务**：
尝试输入以下指令，观察机器人反应：

1.  *"往前走两步"* (预期: MOVE)
2.  *"把前面的红色方块拿起来"* (预期: GRAB)
3.  *"别动！"* (预期: STOP)
4.  *"今天天气不错"* (预期: 置信度低 或 误判，观察 Confidence)

## 3 💥 面试必问：模型上线后太慢怎么办？

**场景**：
面试官问：“黄同学，你的 BERT 模型效果不错，但现在我们要把它部署到机器人的嵌入式芯片（如 Jetson Nano）上，每秒只能跑 3 帧，太慢了，怎么优化？”

**标准回答模板 (由浅入深)**：

1.  **工程层优化 (最快落地)**：

      * **ONNX Runtime**: 将 PyTorch 模型导出为 `.onnx` 格式。ONNX Runtime 对图计算做了大量合并优化，通常能提速 2-5 倍。
      * **TensorRT**: 如果是用 NVIDIA 的芯片（Orin/Jetson），转为 TensorRT 引擎是终极方案，针对硬件层级优化。

2.  **算法层优化 (精度置换速度)**：

      * **量化 (Quantization)**: 目前我们用的是 FP32（32位浮点）。可以转为 **FP16** (半精度) 甚至 **INT8** (8位整数)。模型体积变成 1/4，速度飞快，精度通常只掉一点点。
      * **剪枝 (Pruning)**: 把神经网络中权重接近 0 的连接剪掉，稀疏化模型。

3.  **模型层替换 (架构替换)**：

      * **蒸馏 (Distillation)**: 使用 **DistilBERT** 或 **TinyBERT**。让一个小模型去模仿大模型（Teacher）的行为。DistilBERT 参数量少 40%，速度快 60%，保留 97% 的性能。

**加分操作**:

  * *“我在项目中其实也考虑过这个问题，所以我把 `inference.py` 设计得非常解耦，未来只需要把 `self.model` 替换成 `ort.InferenceSession` 就可以无缝切换到 ONNX Runtime。”*

## 4 ⚔️ 每日算法题 (LeetCode 17)

今天我们做一道跟“映射”有关的经典回溯题目，非常锻炼逻辑。

### 🎯 题目: [LeetCode 17. 电话号码的字母组合 (Letter Combinations of a Phone Number)](../../LeetCode%20practice/1-50.ipynb)

  * **难度**: Medium
  * **标签**: 哈希表、字符串、回溯 (Backtracking)
  * **描述**: 给定一个仅包含数字 `2-9` 的字符串，返回所有它能表示的字母组合。（就像老式手机键盘按键）。
      * 输入: `"23"`
      * 输出: `["ad","ae","af","bd","be","bf","cd","ce","cf"]`
  * **为什么选这题**:
      * 这是标准的 **DFS / 回溯** 模板题。
      * 在具身智能的任务规划（Task Planning）中，经常需要遍历多种可能性的组合路径，这种思维非常重要。

## ✅ Day 5 任务清单

1.  **编写 `src/inference.py`**: 实现 `IntentPredictor` 类。
2.  **交互测试**: 运行 Notebook 中的 `while True` 循环，亲自指挥你的“BERT 机器人”，测试它的反应灵敏度。
3.  **面试准备**: 熟读并理解“推理加速”的三种方案（ONNX, 量化, 蒸馏）。
4.  **算法**: 通过 LeetCode 17。